<a href="https://colab.research.google.com/github/mark-jardine/mark-jardine/blob/main/Rodent_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install pre-requisites
!pip install -Uqq fastbook
!pip install voila
!jupyter serverextension enable --sys prefix voila

In [ ]:
#Imports
from fastbook import *
from fastai.vision.widgets import *

In [ ]:
#Setup fastbook
import fastbook
fastbook.setup_book()

In [ ]:
#Setup azure key(see chapter 2 of fast.ai lectures)
key = os.environ.get("AZURE_SEARCH_KEY","d2cda716193847808439e55b3b879da5")

In [ ]:
#Set up list of rodents + create path
rodent_types = "Rat", "Capybara", "Hamster"
path = Path("rodents")
print(rodent_types)

In [ ]:
#Make directories if they dont exist
if not path.exists():
    path.mkdir()
    for r in rodent_types:
        print(r)
        dest = (path/r)
        dest.mkdir(exist_ok = True)
        results = search_images_bing(key, f'{r} rodent') #Search for images of rodents with azure key
        download_images(dest, urls = results.attrgot('contentUrl')) #download images and store in the path

In [ ]:
path.ls
fns = get_image_files(path)
fns

In [ ]:
#Remove failed images
failed = verify_images(fns)
failed
failed.map(Path.unlink)

In [ ]:
#Create a rodent Datablock object
rodents = DataBlock(
    blocks = (ImageBlock, CategoryBlock),
    get_items = get_image_files,
    splitter = RandomSplitter(valid_pct = 0.2, seed = 42),
    get_y = parent_label,
    item_tfms = Resize(128))
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
#Set up Learner
rodents = rodents.new(item_tfms = RandomResizedCrop(224,min_scale = 0.5),
       batch_tfms = aug_transforms())
dls = rodents.dataloaders(path, num_workers=0)
learn = cnn_learner(dls,resnet34,metrics = error_rate)

#Start transfer learning
learn.fine_tune(4)

In [ ]:
#Set up a cleaner to manually remove high loss images
cleaner = ImageClassifierCleaner(learn)
cleaner